Bonus

As you examine the data, you are overcome with a creeping suspicion that the dataset is fake. You surmise that your boss handed you spurious data in order to test the data engineering skills of a new employee. To confirm your hunch, you decide to take the following steps to generate a visualization of the data, with which you will confront your boss:

1. Import the SQL database into Pandas. (Yes, you could read the CSVs directly in Pandas, but you are, after all, trying to prove your technical mettle.) This step may require some research. Feel free to use the code below to get started. Be sure to make any necessary modifications for your username, password, host, port, and database name:

   ```sql
   from sqlalchemy import create_engine
   engine = create_engine('postgresql://localhost:5432/<your_db_name>')
   connection = engine.connect()
   ```

* Consult [SQLAlchemy documentation](https://docs.sqlalchemy.org/en/latest/core/engines.html#postgresql) for more information.

* If using a password, do not upload your password to your GitHub repository. See [https://www.youtube.com/watch?v=2uaTPmNvH0I](https://www.youtube.com/watch?v=2uaTPmNvH0I) and [https://help.github.com/en/github/using-git/ignoring-files](https://help.github.com/en/github/using-git/ignoring-files) for more information.

2. Create a histogram to visualize the most common salary ranges for employees.

3. Create a bar chart of average salary by title.

See below for importing of the sql file into Pandas and processing the data here within the pandas data file.  All graphs and cleaned tables are exported to "Outputs" folder.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import urllib.parse
import psycopg2
import pandas.io.sql as psql
from sqlalchemy import create_engine

%matplotlib inline

In [ ]:
# Connection parameters, yours will be different
param_dic = {
    "host"      : "localhost",
    "database"  : "SQL_CHallenge",
    "user"      : "postgres",
    "password"  : "Tic@tac56"
}

def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

In [ ]:
def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df
    print(df)

In [ ]:
conn = connect(param_dic)
column_names = ["emp_no", "salary"]

# Execute the "SELECT *" query
salaries_df = postgresql_to_dataframe(conn, "SELECT * FROM salaries", column_names)
salaries_df.head()


In [ ]:
#Salaries Dataframe

salaries = pd.read_sql("SELECT * FROM salaries", conn)
salaries.head(10)


In [ ]:
# Renaming Columns for a cleaner look
salaries_clean = salaries.rename(columns={"emp_no": "Employee Number", "salary": "Salary",})
salaries_clean.head(10)

In [ ]:
# Employees Dataframe

employees = pd.read_sql("SELECT * FROM employees", conn)
# employees.head(10)

# Renaming Columns for a cleaner look
employees_clean = employees.rename(columns={"emp_no": "Employee Number","emp_title_id":"Employee Title ID","birth_date": "Birth Date", "first_name" : "First Name", "last_name" : "Last Name", "sex" : "Gender", "hire_date" : "Hire Date"})
employees_clean.head(10)

In [ ]:
# Department Employee Dataframe

dept_employee = pd.read_sql("SELECT * FROM dept_employee", conn)
dept_employee.head(10)

dept_employee_clean = dept_employee.rename(columns={"emp_no": "Employee Number", "dept_no": "Department Number", })
dept_employee_clean.head(10)

In [ ]:
# Departments Dataframe

departments = pd.read_sql("SELECT * FROM departments", conn)
departments.head(10)

departments_clean = departments.rename(columns={"dept_name": "Department Name", "dept_no": "Department Number"})
departments_clean.head(10)

In [ ]:
# Titles Dataframe

titles = pd.read_sql("SELECT * FROM titles", conn)
# titles.head(10)

titles_clean = titles.rename(columns={"title_id":"Employee Title ID", "title": "Title", })
titles_clean.head(10)

In [ ]:
# Merge DF -- Employees and Salaries

employees_salaries = pd.merge(employees_clean, salaries_clean, on="Employee Number", how="inner")
employees_salaries.head(10)

In [ ]:
# Merge DF -- on Employee Number, common -- Salaries and Titles 

salaries_titles = pd.merge(employees_salaries, titles_clean, on="Employee Title ID", how="inner" )

salaries_titles.head(10)

In [ ]:
salaries_titles_grouped_df = salaries_titles.groupby("Title").mean()
salaries_titles_grouped_df

In [ ]:
# Dropping the Employee Number as it's no longer needed, makes it easier to visualize the plots

title_salary_df = salaries_titles_grouped_df.drop(columns = "Employee Number")
title_salary_df = title_salary_df.reset_index()
title_salary_df

In [ ]:
# Salary by Employees

salaries_employees = pd.merge(salaries_clean, employees_clean, on="Employee Number", how="inner")
salaries_employees.head(10)

In [ ]:
# Merge Departments and Department Employee

dept_dept_employee = pd.merge(departments_clean, dept_employee_clean, on="Department Number", how="inner")
dept_dept_employee.head(10)

In [ ]:
# Merge salaries_department to dept_dept_emp

second_merge_on_emp_no = pd.merge(employees_salaries, dept_dept_employee, on="Employee Number", how="inner")
second_merge_on_emp_no.head(10)

In [ ]:
second_merge_on_emp_no.columns

In [ ]:
# Drop and Clean

clean_second_merge = second_merge_on_emp_no.drop(columns = ["Department Number", "Hire Date", "Birth Date"])
clean_second_merge.head()

In [ ]:
# Plotting Salary by Department: Create a histogram to visualize the most common salary ranges for employees.

plt.hist(clean_second_merge["Salary"], alpha=1);

plt.xlabel("\nSALARY RANGE ($)", weight="bold");
plt.ylabel("NUMBER OF EMPLOYEES\n", weight="bold");
plt.title("Salary Range Per Employees in Department\n", weight="bold", size=15)

plt.grid()
plt.tight_layout()

plt.savefig("Outputs/Hist_By_Dept_Salaries_Range.png")

In [ ]:
# Salary by Title: Create a histogram to visualize the most common salary ranges for employees.


plt.hist(salaries_titles["Salary"], color="lightblue", alpha=1);

plt.xlabel("\nSALARY RANGE ($)", weight="bold");
plt.ylabel("NUMBER OF EMPLOYEES\n", weight="bold");
plt.title("Salary for Employee based on Title\n", weight="bold", size=15)

plt.grid()
plt.tight_layout()

plt.savefig("Outputs/Hist_Employee_Salary_Range.png")

In [ ]:
# Create a bar chart of average salary by title.

x_axis = title_salary_df["Title"]
y_axis = title_salary_df["Salary"]

plt.bar(x_axis, y_axis, color='slateblue', alpha=1)

tick_locations = [value for value in x_axis]
plt.xticks(tick_locations, x_axis, rotation=90);

plt.xlabel("\nTITLE", weight="bold")
plt.ylabel("AVERAGE SALARY\n", weight="bold")
plt.title("Average Salary by Title\n", weight="bold", size=15)

plt.grid()
plt.tight_layout()

plt.savefig("Outputs/Bar_Avg_Salary_By_Title.png")

In [ ]:
#printing cleaned tables to csv
salaries_clean.to_csv('Outputs/salaries_clean.csv', index=False)
employees_clean.to_csv('Outputs/employees_clean.csv', index=False)
dept_employee_clean.to_csv('Outputs/dept_employee_clean.csv', index=False)
departments_clean.to_csv('Outputs/departments_clean.csv', index=False)


In [ ]:
#Epilogue
second_merge_on_emp_no.loc[second_merge_on_emp_no["Employee Number"] == 499942]